<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #07



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>

<a id='prob'></a>
## Problem #07

<br>
Facilities <br>
$|I| = 500$ 
<br>
<br>
Clients <br>
$|J| = 500$ 


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [1]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time


# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_Capacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(int(file_list[i+shift]))
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        capacity.append(file_list[i+shift])
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/MR1'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_Capacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))




In [2]:
print(num_facilities, num_customers)
print(len(shipping_cost))

500 500
250000


In [12]:
start = time.time()

# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(fixed_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 250500 rows, 250500 columns and 750000 nonzeros
Model fingerprint: 0x59df93f2
Variable types: 250000 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 6e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 9.902874e+07
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve time: 5.81s
Presolved: 250500 rows, 250500 columns, 750000 nonzeros
Variable types: 250000 continuous, 500 integer (500 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6574900e+02   5.000000e+02   0.000000e+00      8s
   17300    1.2267259e+03   5.000000e+02   0.000000e+00     11s
   47

In [4]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

#m.write('UFLP_07_Simplex.lp')


Build a warehouse at location 265.

Build a warehouse at location 442.

Client 1 receives 100.0 % of its demand  from Warehouse 442 .

Client 2 receives 100.0 % of its demand  from Warehouse 442 .

Client 3 receives 100.0 % of its demand  from Warehouse 442 .

Client 4 receives 100.0 % of its demand  from Warehouse 265 .

Client 5 receives 100.0 % of its demand  from Warehouse 442 .

Client 6 receives 100.0 % of its demand  from Warehouse 442 .

Client 7 receives 100.0 % of its demand  from Warehouse 442 .

Client 8 receives 100.0 % of its demand  from Warehouse 442 .

Client 9 receives 100.0 % of its demand  from Warehouse 265 .

Client 10 receives 100.0 % of its demand  from Warehouse 442 .

Client 11 receives 100.0 % of its demand  from Warehouse 442 .

Client 12 receives 100.0 % of its demand  from Warehouse 442 .

Client 13 receives 100.0 % of its demand  from Warehouse 265 .

Client 14 receives 100.0 % of its demand  from Warehouse 442 .

Client 15 receives 100.0 % of its demand

<br>
<br>

<a id='random'></a>
### Random Heuristic 


In [5]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

start = time.time()

result=[]
#choose the Facility for each customer
for i in range(num_customers):
    result.append((i,random.randint(0, num_facilities-1)))
   

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
#print(result)
#print("xxx")
#print (facs)
facs=list(set(facs))
    
#print("xxx")
print (facs)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in result: 
    totalC += shipping_cost.get(i)
    #print (shipping_cost.get(i))
    
print(totalC)


end = time.time()
print("TIME IS: ",end - start)

#setup_cost
#cost_per_mile*compute_distance(customers[c], facilities[f]) 
#list(set(output))

[0, 2, 5, 6, 7, 8, 9, 11, 13, 15, 16, 18, 19, 22, 28, 29, 31, 32, 33, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 53, 54, 59, 60, 62, 67, 68, 69, 71, 73, 74, 75, 76, 77, 80, 82, 85, 86, 87, 90, 91, 92, 95, 96, 98, 99, 100, 101, 103, 107, 109, 110, 114, 116, 119, 120, 121, 124, 125, 126, 127, 128, 129, 130, 133, 135, 138, 141, 142, 144, 145, 146, 147, 148, 150, 151, 152, 154, 156, 157, 158, 159, 161, 162, 165, 166, 167, 168, 169, 170, 171, 173, 174, 176, 177, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 193, 196, 198, 199, 200, 201, 203, 205, 206, 209, 210, 212, 213, 214, 217, 219, 220, 221, 222, 224, 226, 227, 230, 231, 232, 233, 234, 235, 237, 238, 239, 241, 243, 245, 246, 247, 248, 249, 251, 253, 255, 256, 257, 260, 261, 262, 263, 264, 265, 266, 269, 270, 271, 272, 274, 276, 279, 280, 282, 283, 285, 286, 287, 288, 290, 293, 294, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 311, 312, 313, 314, 317, 318, 319, 321, 322, 323, 324, 325, 326, 328, 332, 334, 33

In [6]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 1.

Build a warehouse at location 3.

Build a warehouse at location 6.

Build a warehouse at location 7.

Build a warehouse at location 8.

Build a warehouse at location 9.

Build a warehouse at location 10.

Build a warehouse at location 12.

Build a warehouse at location 14.

Build a warehouse at location 16.

Build a warehouse at location 17.

Build a warehouse at location 19.

Build a warehouse at location 20.

Build a warehouse at location 23.

Build a warehouse at location 29.

Build a warehouse at location 30.

Build a warehouse at location 32.

Build a warehouse at location 33.

Build a warehouse at location 34.

Build a warehouse at location 38.

Build a warehouse at location 39.

Build a warehouse at location 40.

Build a warehouse at location 43.

Build a warehouse at location 44.

Build a warehouse at location 45.

Build a warehouse at location 46.

Build a warehouse at location 47.

Build a warehouse at location 48.

Build a warehouse at loca

In [7]:
len(facs)

304

<br>
<br>

<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [8]:
#Get the lowest shipping cost for each Client

start = time.time()

#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

#print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[3, 4, 5, 6, 7, 9, 16, 17, 19, 20, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 40, 44, 47, 48, 49, 52, 53, 54, 56, 61, 67, 69, 70, 71, 73, 74, 75, 77, 78, 81, 82, 83, 84, 85, 86, 87, 88, 92, 94, 95, 97, 98, 100, 101, 102, 103, 104, 105, 106, 108, 111, 114, 116, 117, 119, 122, 123, 124, 125, 126, 127, 129, 130, 131, 136, 138, 139, 140, 142, 146, 147, 148, 149, 152, 153, 154, 156, 157, 158, 159, 160, 162, 163, 166, 167, 168, 169, 170, 171, 172, 173, 174, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 197, 198, 202, 203, 204, 205, 207, 210, 212, 213, 215, 216, 219, 221, 222, 223, 224, 225, 226, 227, 229, 230, 233, 234, 235, 237, 239, 240, 241, 242, 244, 249, 251, 252, 253, 254, 256, 259, 260, 261, 262, 263, 264, 267, 268, 270, 272, 275, 276, 277, 279, 280, 281, 283, 285, 286, 289, 293, 295, 296, 297, 300, 302, 303, 304, 305, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 325, 326, 329, 330, 331, 332, 333, 335, 336, 337, 

<br>
<br>

<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [9]:
#Get the greatest shipping cost for each Client

start = time.time()


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) 
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i,j)):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[0, 1, 3, 4, 5, 6, 7, 8, 11, 12, 13, 15, 17, 20, 21, 22, 24, 25, 27, 28, 31, 35, 36, 37, 39, 40, 41, 43, 45, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 99, 100, 101, 102, 106, 107, 109, 111, 112, 113, 115, 118, 119, 120, 121, 124, 125, 128, 130, 134, 135, 136, 137, 140, 141, 142, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 160, 162, 163, 168, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 186, 188, 189, 190, 191, 192, 195, 196, 197, 199, 201, 204, 208, 209, 212, 214, 215, 216, 217, 218, 219, 220, 223, 225, 227, 231, 232, 233, 234, 235, 236, 239, 241, 243, 244, 245, 253, 255, 257, 259, 262, 263, 264, 265, 266, 267, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 282, 283, 284, 286, 290, 291, 292, 293, 295, 297, 300, 306, 307, 309, 310, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 328, 330, 331, 332, 

<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [10]:
#Get the lowest shipping cost for each Client


start = time.time()


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[264]
100
54501.779203999984
1
TIME IS:  0.39223694801330566


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [11]:
#Get the lowest shipping cost for each Client

start = time.time()


#choose the lowest cost 
path={}
for i in range(num_customers):
    aux={}
    aux_key=()
    for j in range(num_facilities):

        #empty list for the Client
        if not aux:
            aux[(i,j)] = shipping_cost.get((i,j)) + fixed_cost[j]
            aux_key = ((i,j)) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i,j)) + fixed_cost[j]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i,j)] = shipping_cost.get((i,j))+ fixed_cost[j]             
            aux_key = ((i,j))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

#print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
#print (facs)
facs=list(set(facs))
print (facs)

totalC=0
for w in facs:
    totalC += fixed_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    #print(i)
    totalC += i 
    
print(totalC)
print (len(facs))


end = time.time()
print("TIME IS: ",end - start)

[47]
596394
298797280.75868505
1
TIME IS:  0.31169581413269043
